<a href="https://colab.research.google.com/github/phillippfarias/SIMULADOR-LIMITES-DE-GASTOS-COM-PESSOAL-LRF/blob/main/Conhe%C3%A7a_o_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Célula 1 — Instala dependências e cloudflared
!pip install streamlit pandas plotly openpyxl -q
# Baixa e instala cloudflared (usado para gerar link público .trycloudflare.com)
!wget -q -O cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared.deb || apt-get -y -qq -f install


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 131.4 MB/s eta 0:00:00
Selecting previously unselected package cloudflared.
(Reading database ... 126374 files and directories currently installed.)
Preparing to unpack cloudflared.deb ...
Unpacking cloudflared (2025.8.1) ...
Setting up cloudflared (2025.8.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import io
import plotly.graph_objs as go

# ---------- Config / Título ----------
st.set_page_config(page_title="Simulador de Despesa com Pessoal (LRF)", layout="wide")
st.title("📊 Simulador de Despesa com Pessoal (LRF) - Limites Máximo/Prudencial/Alerta")

# ---------- Utilitários ----------
def fmt_r(x):
    try:
        return f"R$ {x:,.2f}"
    except:
        return "-"

def calc_limits(rcl_adj, max_pct, prud_factor, alert_factor):
    limite_max = rcl_adj * max_pct
    limite_prud = limite_max * prud_factor
    limite_alert = limite_max * alert_factor
    return limite_max, limite_prud, limite_alert

def compute_adjustments(rcl, desp, max_pct, prud_factor, alert_factor):
    """Para cada limite retorna redução necessária (R$ e %) e aumento de RCL necessário (R$ e %)"""
    res = {}
    nomes = ["Máximo","Prudencial","Alerta"]
    fatores = [1.0, prud_factor, alert_factor]
    for nome, f in zip(nomes, fatores):
        limite = rcl * max_pct * f
        if desp > limite:
            reduce_R = desp - limite
            reduce_pct = (reduce_R / desp * 100) if desp else np.nan
        else:
            reduce_R = 0.0
            reduce_pct = 0.0
        denom = max_pct * f
        if denom > 0:
            rcl_needed = desp / denom
            rcl_increase_R = max(0.0, rcl_needed - rcl)
            rcl_increase_pct = (rcl_increase_R / rcl * 100) if rcl else np.nan
        else:
            rcl_increase_R = np.nan
            rcl_increase_pct = np.nan
        res[nome] = {
            "limite": limite,
            "reduce_R": reduce_R,
            "reduce_pct": reduce_pct,
            "rcl_increase_R": rcl_increase_R,
            "rcl_increase_pct": rcl_increase_pct
        }
    return res

# ---------- Sidebar: upload ou manual (valores padrão vindos da imagem) ----------
st.sidebar.header("⚙️ Entradas / Upload")

uploaded = st.sidebar.file_uploader("📥 (Opcional) Upload XLSX com RCL/Despesa (colunas: RCL_adjusted, Despesa_Pessoal)", type=["xlsx"])
if uploaded:
    try:
        df_up = pd.read_excel(uploaded)
        # tenta mapear nomes de coluna
        if "RCL_adjusted" in df_up.columns:
            rcl_default = float(df_up["RCL_adjusted"].iloc[0])
        elif "RCL ajustada" in df_up.columns:
            rcl_default = float(df_up["RCL ajustada"].iloc[0])
        else:
            rcl_default = 36273923688.14
        if "Despesa_Pessoal" in df_up.columns:
            desp_default = float(df_up["Despesa_Pessoal"].iloc[0])
        elif "Despesa com Pessoal" in df_up.columns:
            desp_default = float(df_up["Despesa com Pessoal"].iloc[0])
        else:
            desp_default = 15127218477.20
        st.sidebar.success("Planilha carregada — valores usados como padrão.")
    except Exception as e:
        rcl_default = 36273923688.14
        desp_default = 15127218477.20
        st.sidebar.error("Erro ao ler XLSX — usando valores padrão.")
else:
    # Valores padrao conforme imagem enviada
    # RCL (ajustada para cálculo): 36.273.923.688,14
    # Despesa com pessoal (DTP): 15.127.218.477,20
    rcl_default = 36273923688.14
    desp_default = 15127218477.20

st.sidebar.subheader("Parâmetros Atuais")
rcl_atual = st.sidebar.number_input("RCL ajustada (Atual) (R$)", value=rcl_default, format="%.2f", min_value=0.0)
desp_atual = st.sidebar.number_input("Despesa com Pessoal (Atual) (R$)", value=desp_default, format="%.2f", min_value=0.0)

st.sidebar.markdown("---")
st.sidebar.subheader("Parâmetros dos Limites (padrão da imagem)")
max_pct = st.sidebar.number_input("Limite Máximo (% RCL)", value=0.49, min_value=0.0, max_value=1.0, step=0.01, format="%.2f")
prud_factor = st.sidebar.number_input("Fator Prudencial (fração do máximo)", value=0.95, min_value=0.0, max_value=1.0, step=0.01, format="%.2f")
alert_factor = st.sidebar.number_input("Fator Alerta (fração do máximo)", value=0.90, min_value=0.0, max_value=1.0, step=0.01, format="%.2f")

st.sidebar.markdown("---")
st.sidebar.subheader("🎯 Simulação (Cenário Simulado)")
sim_type = st.sidebar.selectbox("Tipo de simulação", (
    "Nenhuma",
    "Aumento despesa (%)", "Aumento despesa (R$)",
    "Redução despesa (%)", "Redução despesa (R$)",
    "Aumento receita (%)", "Aumento receita (R$)",
    "Redução receita (%)", "Redução receita (R$)"
))
sim_val = st.sidebar.number_input("Valor da simulação (percentual ou R$)", value=0.0, format="%.2f")

# ---------- Calcula cenário Atual e Simulado ----------
rcl = {"atual": float(rcl_atual), "sim": float(rcl_atual)}
desp = {"atual": float(desp_atual), "sim": float(desp_atual)}

if sim_type == "Aumento despesa (%)":
    desp["sim"] = desp_atual * (1 + sim_val/100.0)
elif sim_type == "Aumento despesa (R$)":
    desp["sim"] = desp_atual + sim_val
elif sim_type == "Redução despesa (%)":
    desp["sim"] = desp_atual * (1 - sim_val/100.0)
elif sim_type == "Redução despesa (R$)":
    desp["sim"] = max(0.0, desp_atual - sim_val)
elif sim_type == "Aumento receita (%)":
    rcl["sim"] = rcl_atual * (1 + sim_val/100.0)
elif sim_type == "Aumento receita (R$)":
    rcl["sim"] = rcl_atual + sim_val
elif sim_type == "Redução receita (%)":
    rcl["sim"] = rcl_atual * (1 - sim_val/100.0)
elif sim_type == "Redução receita (R$)":
    rcl["sim"] = max(0.0, rcl_atual - sim_val)

# limites Atual e Simulado
lim_atual = calc_limits(rcl["atual"], max_pct, prud_factor, alert_factor)
lim_sim = calc_limits(rcl["sim"], max_pct, prud_factor, alert_factor)

# ajustes (para o cenário simulado)
ajustes = compute_adjustments(rcl["sim"], desp["sim"], max_pct, prud_factor, alert_factor)

# ---------- Visão rápida (Gauge + Ajustes compactos) ----------
st.header("📌 Visão rápida")
col_g1, col_g2 = st.columns([1.6, 1])

pct_atual = (desp["atual"] / lim_atual[0] * 100) if lim_atual[0] else np.nan
pct_sim = (desp["sim"] / lim_sim[0] * 100) if lim_sim[0] else np.nan

with col_g1:
    fig_g = go.Figure(go.Indicator(
        mode="gauge+number+delta",
        value=round(pct_sim,2) if not np.isnan(pct_sim) else 0,
        delta={'reference': round(pct_atual,2) if not np.isnan(pct_atual) else 0},
        title={'text': "Simulado % do Limite Máximo"},
        gauge={
            'axis': {'range': [0, 120]},
            'bar': {'color': "orange"},
            'steps': [
                {'range': [0, 90], 'color': "lightgreen"},
                {'range': [90, 95], 'color': "yellow"},
                {'range': [95, 120], 'color': "red"}
            ]
        }
    ))
    fig_g.update_layout(height=300)
    st.plotly_chart(fig_g, use_container_width=True)

with col_g2:
    st.markdown("### 🔧 Ajustes necessários (Cenário Simulado)")
    rows = []
    for nome in ["Máximo","Prudencial","Alerta"]:
        a = ajustes[nome]
        # compactar texto sem poluir
        reduce_txt = f"{fmt_r(a['reduce_R'])} ({a['reduce_pct']:.2f}%)" if a['reduce_R']>0 else "—"
        rcl_txt = f"{fmt_r(a['rcl_increase_R'])} ({a['rcl_increase_pct']:.2f}%)" if a['rcl_increase_R']>0 else "—"
        rows.append({"Limite": nome, "Reduzir Despesa (R$ / %)": reduce_txt, "Aumentar RCL (R$ / %)": rcl_txt})
    st.table(pd.DataFrame(rows).set_index("Limite"))

st.markdown("---")

# ---------- Dashboards — todos na tela inicial ----------
st.header("📊 Dashboards — Atual vs Simulado")

# 1) Limites: comparação Atual vs Simulado (horizontal grouped bars)
lim_df = pd.DataFrame({
    "Limite": ["Máximo","Prudencial","Alerta"],
    "Atual": list(lim_atual),
    "Simulado": list(lim_sim)
})
fig_lim = go.Figure()
fig_lim.add_trace(go.Bar(x=lim_df["Atual"], y=lim_df["Limite"], orientation='h', name="Atual", marker=dict(color="lightgray")))
fig_lim.add_trace(go.Bar(x=lim_df["Simulado"], y=lim_df["Limite"], orientation='h', name="Simulado", marker=dict(color="darkgray")))
fig_lim.update_layout(height=320, barmode='group', xaxis_title="R$ (reais)")
st.plotly_chart(fig_lim, use_container_width=True)
st.caption("Comparação dos limites (Atual vs Simulado).")

# 2) Despesa: Atual vs Simulado (barra simples)
desp_df = pd.DataFrame({
    "Cenário": ["Atual","Simulado"],
    "Despesa": [desp["atual"], desp["sim"]]
})
fig_d = go.Figure()
fig_d.add_trace(go.Bar(x=desp_df["Cenário"], y=desp_df["Despesa"], marker_color=["#1f77b4","#ff7f0e"]))
fig_d.update_layout(height=260, yaxis_title="R$ (reais)")
st.plotly_chart(fig_d, use_container_width=True)
st.caption(f"Para o Limite Máximo (Simulado): Redução necessária = {fmt_r(ajustes['Máximo']['reduce_R'])} ({ajustes['Máximo']['reduce_pct']:.2f}%)" if ajustes['Máximo']['reduce_R']>0 else "Para Limite Máximo: redução não necessária.")

# 3) Margens por limite (comparação Atual vs Simulado)
margens = {
    "Limite": ["Máx","Prud","Alerta"],
    "Margem_Atual": [lim_atual[0]-desp["atual"], lim_atual[1]-desp["atual"], lim_atual[2]-desp["atual"]],
    "Margem_Sim": [lim_sim[0]-desp["sim"], lim_sim[1]-desp["sim"], lim_sim[2]-desp["sim"]]
}
marg_df = pd.DataFrame(margens)
fig_m = go.Figure()
fig_m.add_trace(go.Bar(y=marg_df["Limite"], x=marg_df["Margem_Atual"], orientation='h', name="Margem Atual", marker=dict(color="lightblue")))
fig_m.add_trace(go.Bar(y=marg_df["Limite"], x=marg_df["Margem_Sim"], orientation='h', name="Margem Simulado", marker=dict(color="lightcoral")))
fig_m.update_layout(barmode='group', height=320, xaxis_title="Margem (R$)")
st.plotly_chart(fig_m, use_container_width=True)
st.caption("Margens (diferença entre limite e despesa). Valores negativos indicam extrapolação.")

# 4) Receita x Despesa (scatter) com linhas de limite (do cenário simulado)
fig_sc = go.Figure()
fig_sc.add_trace(go.Scatter(x=[rcl["atual"]], y=[desp["atual"]], mode="markers+text", text=["Atual"], textposition="top center", marker=dict(size=12, color="#1f77b4"), name="Atual"))
fig_sc.add_trace(go.Scatter(x=[rcl["sim"]], y=[desp["sim"]], mode="markers+text", text=["Simulado"], textposition="top center", marker=dict(size=12, color="#ff7f0e"), name="Simulado"))
# linhas horizontais dos limites (simulado)
fig_sc.add_hline(y=lim_sim[0], line=dict(color="red", dash="dash"), annotation_text="Limite Máx (Sim.)", annotation_position="bottom right")
fig_sc.add_hline(y=lim_sim[1], line=dict(color="orange", dash="dot"), annotation_text="Limite Prud (Sim.)", annotation_position="bottom right")
fig_sc.add_hline(y=lim_sim[2], line=dict(color="green", dash="dot"), annotation_text="Limite Alerta (Sim.)", annotation_position="bottom right")
fig_sc.update_layout(title="Receita x Despesa (pontos Atual & Simulado)", xaxis_title="RCL Ajustada (R$)", yaxis_title="Despesa com Pessoal (R$)", height=420)
st.plotly_chart(fig_sc, use_container_width=True)
st.caption("Pontos indicam posição Atual e Simulado; linhas horizontais mostram limites do cenário simulado.")

st.markdown("---")

# ---------- Tabela detalhada (Atual / Simulado) ----------
st.header("📋 Tabela detalhada (Atual / Simulado)")
rows = []
for key in ["Atual","Simulado"]:
    Lm, Lp, La = (lim_atual if key=="Atual" else lim_sim)
    D = (desp["atual"] if key=="Atual" else desp["sim"])
    R = (rcl["atual"] if key=="Atual" else rcl["sim"])
    rows.append({
        "Cenário": key,
        "RCL ajustada (R$)": R,
        "Despesa Pessoal (R$)": D,
        "Limite Máx (R$)": Lm,
        "Limite Prud (R$)": Lp,
        "Limite Alerta (R$)": La,
        "Margem Máx (R$)": Lm - D,
        "Margem Prud (R$)": Lp - D,
        "Margem Alerta (R$)": La - D,
        "% Ocup Máx": (D / Lm * 100) if Lm else np.nan,
        "% Ocup Prud": (D / Lp * 100) if Lp else np.nan,
        "% Ocup Alerta": (D / La * 100) if La else np.nan,
    })
table_df = pd.DataFrame(rows)
fmt_cols = {
    "RCL ajustada (R$)": "{:,.2f}",
    "Despesa Pessoal (R$)": "{:,.2f}",
    "Limite Máx (R$)": "{:,.2f}",
    "Limite Prud (R$)": "{:,.2f}",
    "Limite Alerta (R$)": "{:,.2f}",
    "Margem Máx (R$)": "{:,.2f}",
    "Margem Prud (R$)": "{:,.2f}",
    "Margem Alerta (R$)": "{:,.2f}",
    "% Ocup Máx": "{:.2f}%",
    "% Ocup Prud": "{:.2f}%",
    "% Ocup Alerta": "{:.2f}%"
}
st.dataframe(table_df.style.format(fmt_cols), height=240)

st.markdown("---")
st.subheader("📥 Exportar tabela")
csv_buf = io.StringIO()
table_df.to_csv(csv_buf, index=False, sep=";")
st.download_button("Baixar CSV (Detalhado)", csv_buf.getvalue().encode("utf-8"), file_name="margens_detalhado.csv", mime="text/csv")

st.caption("Notas: 'Atual' = situação atual; 'Simulado' = após aplicação da simulação escolhida. Ajustes sugeridos são valores para que a despesa passe a respeitar o limite.")


Writing app.py


In [4]:
# Célula 3 corrigida — inicia Streamlit e captura o link público do cloudflared
import subprocess, time, sys, re

# 1) Inicia Streamlit em background (stdout será capturado)
streamlit_cmd = [sys.executable, "-m", "streamlit", "run", "app.py",
                 "--server.port=8501", "--server.address=0.0.0.0"]
print("Iniciando Streamlit...")
st_proc = subprocess.Popen(streamlit_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Dá uma pequena pausa para o Streamlit começar
time.sleep(4)

# 2) Inicia cloudflared (quick tunnel) e lê os logs em busca do link público
print("Iniciando cloudflared (criando túnel)...")
cf_proc = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:8501", "--no-autoupdate"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

# Regex para capturar o link trycloudflare
pattern = re.compile(r'https?://[^\s]*trycloudflare\.com[^\s]*', re.IGNORECASE)

url = None
start = time.time()
timeout = 60  # segundos para procurar o link automaticamente

print("Aguardando aparecer o link público (pode levar alguns segundos)...\n(Se nada aparecer após ~60s, verifique os logs abaixo.)\n")

# Lê linhas do stdout do cloudflared até achar o link ou timeout
while True:
    line = cf_proc.stdout.readline()
    if line:
        print(line.strip())  # mostra logs úteis para diagnóstico
        m = pattern.search(line)
        if m:
            url = m.group(0)
            print("\n>>> LINK PÚBLICO ENCONTRADO:", url)
            break
    else:
        # se não há nova linha, verifica se o processo terminou
        if cf_proc.poll() is not None:
            print("\ncloudflared terminou (verifique logs acima).")
            break
        # timeout para não ficar preso indefinidamente
        if time.time() - start > timeout:
            print("\nTimeout: não encontrei o link em ~60s. Verifique os logs acima.")
            break
        time.sleep(0.2)

if url:
    print("\nAbra o link acima no navegador (PC ou celular).")
else:
    print("\nSe não encontrou o link, veja a saída de logs acima para buscar manualmente uma URL do tipo https://xxxx.trycloudflare.com")

print("\nProcessos em execução:")
print(f" - streamlit PID = {st_proc.pid}")
print(f" - cloudflared PID = {cf_proc.pid}")

print("\nPara encerrar manualmente, execute em outra célula:")
print(f"!kill {st_proc.pid}  # para parar o Streamlit")
print(f"!kill {cf_proc.pid}  # para parar o cloudflared")


Iniciando Streamlit...
Iniciando cloudflared (criando túnel)...
Aguardando aparecer o link público (pode levar alguns segundos)...
(Se nada aparecer após ~60s, verifique os logs abaixo.)

2025-09-10T17:11:10Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-10T17:11:10Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-09-10T17:11:13Z INF +--------------------------------------------------------------------------------------------+
2025-09-10T17